In [3]:
import numpy as np
import pandas as pd

In [4]:
steam_filepath = "data/steam_users.csv"
stream_df = pd.read_csv(steam_filepath)

In [5]:
stream_df.head()

,user,game,purchase_play,hours,others
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0


In [6]:
print("Number of games : {0}".format(len(stream_df.game.unique())))
print("Number of users : {0}".format(len(stream_df.user.unique())))

Number of games : 5155
Number of users : 12393


In [7]:
# Dropping the others column as it has zero values for all
stream_df.drop(columns=["others"],inplace=True,axis=1)

In [8]:
stream_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   user           200000 non-null  int64  
 1   game           200000 non-null  object 
 2   purchase_play  200000 non-null  object 
 3   hours          200000 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.1+ MB


In [9]:
# Check if any column has Null value
print("Before check - shape: ", stream_df.shape)
stream_df.dropna(how='any', inplace=True)
print("Before check - shape: ", stream_df.shape)

Before check - shape:  (200000, 4)
Before check - shape:  (200000, 4)


In [10]:
# Check if any games for duplicate purchase and remove it.
stream_df.drop_duplicates(subset=['user','game','purchase_play'], keep='first', inplace=True)
stream_df.shape

(199281, 4)

#### Let's do Feature Engineering 

I am dividing the purchase / play column into 2 columns. Data collection is basically duplicating each record with one labeled as ‘purchase’ and one labelled as ‘play’. The ‘purchase row’ records 1 hour. Obviously this doesn’t make sense, so I’ll remove this in the split and keep only the ‘play‘ hours.

In [11]:
def purchase_split(data):
    if data['purchase_play'] == 'purchase':
        return 1
    else:
        return 0
    
def play_split(data):
    if data['purchase_play'] == 'play':
        return 1
    else:
        return 0
    
def play_hours_split(data):
    return data['hours'] - data['purchase']

In [12]:
stream_df['purchase'] = stream_df.apply (lambda data: purchase_split(data), axis=1)
stream_df['play'] = stream_df.apply (lambda data: play_split(data), axis=1)
stream_df['hours'] = stream_df.apply (lambda data: play_hours_split(data), axis=1)
df_cleaned = stream_df.groupby(['user','game']).sum().reset_index()

In [13]:
df_cleaned

,user,game,purchase_play,hours,purchase,play
0,5250,Alien Swarm,purchaseplay,4.9,1,1
1,5250,Cities Skylines,purchaseplay,144.0,1,1
2,5250,Counter-Strike,purchase,0.0,1,0
3,5250,Counter-Strike Source,purchase,0.0,1,0
4,5250,Day of Defeat,purchase,0.0,1,0
...,...,...,...,...,...,...
128799,309626088,Age of Empires II HD Edition,purchaseplay,6.7,1,1
128800,309812026,Counter-Strike Nexon Zombies,purchase,0.0,1,0
128801,309812026,Robocraft,purchase,0.0,1,0
128802,309824202,Dota 2,purchaseplay,0.7,1,1


In [14]:
# Let's save the cleaned dataset
cleaned_dataset_filepath = "data/steam_200_cleaned_data.csv"
df_cleaned.to_csv(cleaned_dataset_filepath,index=False,header=True)